In [197]:
import csv
import random

In [198]:
# candidate info
candidate_count = 0
candidates = []

# ballot mete data
total_voter_count = 0
total_vote_count = 0
unique_order_count = 0

# ballot data
# map a unique order of ballots to the total count of ballots in this order
 # format: tuple(first choice,second,third choice...): count
ballots = {}

with open("data/dublin-north.soi", newline="\n") as data_file:
    data = csv.reader(data_file)
    i = 0
    for row in data:
        if i == 0:
            candidate_count = int(row[0])
        elif i <= candidate_count:
            candidates.append(row[1])
        elif i == candidate_count + 1:
            total_voter_count = int(row[0])
            total_vote_count = int(row[1])
            unique_order_count = int(row[2])
        else:
            ballot = tuple(int(j) for j in row[1:])
            ballots[ballot] = int(row[0])
        i += 1

In [199]:
# print(candidate_count)
# for key in ballots.keys():
#     print(key, ballots[key])
#     print(len(key))
# for ballot in ballots:
#     print(ballot, ballots[ballot])

In [200]:
# train model function

In [201]:
# Return an order of ballot as a tuple
def predict(ballots, target_ballot, model):
    # finish this
    
    ##################### place holder ####################
    temp = list(target_ballot)
    temp.append(random.randint(1,12))
    improved_target_ballot = tuple(temp)
    #######################################################
    
    return improved_target_ballot

In [202]:
# this code will modify the data in "ballots". Need to rerun above data reading code to restore
# data in the dictionary.
for i in range(1, candidate_count+1):
    # train model for predicting the i+1 preference
    model = None
    
    # update ballots with only i preference
    keys = list(ballots.keys())
    for ballot in keys:
        if len(ballot) == i and ballots[ballot] != 0:
            improved_ballot = predict(ballots, ballot, model)
            if improved_ballot in ballots:
                ballots[improved_ballot] += ballots[ballot]
            else:
                ballots[improved_ballot] = ballots[ballot]
            del ballots[ballot]
#             ballots[ballot] = 0 # set old count to 0, should then have no impact on final vote count
# print(ballots)

In [203]:
# Return a dictionary mapping candidate number to score.
# Calculate candidate scores based on Borda's Rule.
def borda_rule_result(ballots):
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
#     print(candidate_scores)
    for ballot in ballots:
        count = ballots[ballot]
        for i in range(candidate_count):
            candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count
    return candidate_scores

In [204]:
print(borda_rule_result(ballots))

{1: 204226, 2: 278355, 3: 160689, 4: 300808, 5: 176094, 6: 281005, 7: 257334, 8: 145494, 9: 320799, 10: 349792, 11: 124950, 12: 300626}
